In [1]:
data_dir = "/Volumes/KeithSSD/ChesapeakeMicrobiome/data/environmental_raw_data"

columns_desired = c('Station', 'WaterColumnPosition', 'DO')
orig_data <- read.delim(paste(data_dir, 'merged_paired_water_quality_data.txt', sep="/"), 
                        row.names=1)[,columns_desired]
env_data <- orig_data[!is.na(orig_data$DO),]

In [2]:
env_data[,'hypoxia'] <- factor(ifelse(env_data$DO<=2.0,1,0))

In [3]:
table(env_data$hypoxia, env_data$Station)

   
    CB2.2 CB3.1 CB3.2 CB3.3C CB4.1C CB4.2C CB4.3C CB4.4 CB5.1 CB5.2 CB5.3 CB5.4
  0    15     6     2      8      0      3      3    10     6     2    17    18
  1     0     3     6     10      3      6      9     6     3     1     1     0
   
    CB6.1 CB6.2 CB6.3 CB6.4 CB7.1 CB7.2 CB7.3 CB7.4
  0    11    18    17     4    16    13    13     3
  1     0     0     0     0     0     0     0     0

In [4]:
suppressPackageStartupMessages(library(zCompositions))
suppressPackageStartupMessages(library(compositions))

data_objects = list()
rar_df = read.delim('../data/otu_tables/final_rarefied_table.best.txt', row.names=1, sep="\t")[rownames(env_data),]
rar_df = rar_df[which(rowSums(rar_df) > 0), which(colSums(rar_df) > 0)]
col_ordered = intersect(paste('OTU', 1:56832, sep=""), colnames(rar_df))
dim(rar_df)
rar_df = rar_df[,c(col_ordered)]
dim(rar_df)

[1]  233 1349

[1]  233 1349

In [5]:
ra_rar_df = data.matrix(clr(cmultRepl((rar_df), method="CZM", output="p-counts")))
data_objects[['Rare.ASVs']] = ra_rar_df

In [6]:
oeu_file = '../data/oeu_clusters/oeu_abundances.txt'
fxn_file = "../data/faprotax_data/function_abundances.txt"
taxa_file = '../data/otu_tables/taxa_family_abundances.txt'

data_files = list('Taxa.Family'=taxa_file, 'Fxn.SILVA'=fxn_file, 'ASV.OEU'=oeu_file)

for (i in names(data_files)){
    temp_frame = read.delim(data_files[[i]], row.names=1)[rownames(env_data),]
    data_objects[[i]] = temp_frame[which(rowSums(temp_frame) > 0), which(colSums(temp_frame) > 0)]
    print(colnames(data_objects[[i]])[1:3])
    print(c(dim(data_objects[[i]]), dim(temp_frame)))
    rowsums_ofThis = (1/(rowSums(data_objects[[i]]) / max(rowSums(data_objects[[i]]))))
    data_objects[[i]] = data_objects[[i]]*rowsums_ofThis
}

[1] "Actinomarinales.Actinomarinaceae"      
[2] "Oceanospirillales.Halomonadaceae"      
[3] "Betaproteobacteriales.Burkholderiaceae"
[1] 233 477 233 477
[1] "methanotrophy"                                        
[2] "acetoclastic_methanogenesis"                          
[3] "methanogenesis_by_disproportionation_of_methyl_groups"
[1] 233  69 233  91
[1] "OEU.1" "OEU.2" "OEU.3"
[1] 233 128 233 128


In [7]:
colnames(data_objects$Fxn.SILVA)

[1] "methanotrophy"                                          
 [2] "acetoclastic_methanogenesis"                            
 [3] "methanogenesis_by_disproportionation_of_methyl_groups"  
 [4] "methanogenesis_using_formate"                           
 [5] "methanogenesis_by_CO2_reduction_with_H2"                
 [6] "methanogenesis_by_reduction_of_methyl_compounds_with_H2"
 [7] "hydrogenotrophic_methanogenesis"                        
 [8] "methanogenesis"                                         
 [9] "methanol_oxidation"                                     
[10] "methylotrophy"                                          
[11] "aerobic_ammonia_oxidation"                              
[12] "aerobic_nitrite_oxidation"                              
[13] "nitrification"                                          
[14] "sulfate_respiration"                                    
[15] "sulfur_respiration"                                     
[16] "dark_sulfite_oxidation"                                 
[17] "sulfite_respiration"                                    
[18] "thiosulfate_respiration"                                
[19] "respiration_of_sulfur_compounds"                        
[20] "anammox"                                                
[21] "nitrate_denitrification"                                
[22] "nitrite_denitrification"                                
[23] "nitrous_oxide_denitrification"                          
[24] "denitrification"                                        
[25] "chitinolysis"                                           
[26] "dark_hydrogen_oxidation"                                
[27] "nitrogen_fixation"                                      
[28] "nitrite_respiration"                                    
[29] "cellulolysis"                                           
[30] "xylanolysis"                                            
[31] "dark_sulfide_oxidation"                                 
[32] "dark_sulfur_oxidation"                                  
[33] "dark_thiosulfate_oxidation"                             
[34] "dark_oxidation_of_sulfur_compounds"                     
[35] "manganese_oxidation"                                    
[36] "manganese_respiration"                                  
[37] "fermentation"                                           
[38] "aerobic_chemoheterotrophy"                              
[39] "human_pathogens_septicemia"                             
[40] "human_pathogens_all"                                    
[41] "human_gut"                                              
[42] "mammal_gut"                                             
[43] "animal_parasites_or_symbionts"                          
[44] "aromatic_hydrocarbon_degradation"                       
[45] "aromatic_compound_degradation"                          
[46] "aliphatic_non_methane_hydrocarbon_degradation"          
[47] "hydrocarbon_degradation"                                
[48] "dark_iron_oxidation"                                    
[49] "iron_respiration"                                       
[50] "nitrate_respiration"                                    
[51] "nitrate_reduction"                                      
[52] "nitrogen_respiration"                                   
[53] "fumarate_respiration"                                   
[54] "intracellular_parasites"                                
[55] "chlorate_reducers"                                      
[56] "predatory_or_exoparasitic"                              
[57] "chloroplasts"                                           
[58] "cyanobacteria"                                          
[59] "anoxygenic_photoautotrophy_H2_oxidizing"                
[60] "anoxygenic_photoautotrophy_S_oxidizing"                 
[61] "anoxygenic_photoautotrophy_Fe_oxidizing"                
[62] "anoxygenic_photoautotrophy"                             
[63] "oxygenic_photoautotrophy"                               
[64] "photoautotrophy"          

In [7]:
to_drop = c('methylotrophy', 'aerobic_ammonia_oxidation', 'sulfate_respiration',
            'dark_sulfite_oxidation', 'arsenate_respiration', 'nitrite_ammonification',
            'dissimilatory_arsenate_reduction', 'nitrite_denitrification', 
            'nitrous_oxide_denitrification', 'nitrate_denitrification', 'fumarate_respiration',
            'mammal_gut', 'plant_pathogen',  'cyanobacteria', 'phototrophy', 
            'anoxygenic_photoautotrophy_S_oxidizing', 'anoxygenic_photoautotrophy_Fe_oxidizing',
            'nitrate_respiration', 'aliphatic_non_methane_hydrocarbon_degradation', 
            'aerobic_chemoheterotrophy', 'nitrite_respiration', 'dark_sulfide_oxidation')

data_objects[['Taxa.Family']] = data.matrix(clr(data_objects[['Taxa.Family']] + 0.1))
data_objects[['ASV.OEU']] = data.matrix(clr(data_objects[['ASV.OEU']] + 0.1))
data_objects[['Fxn.SILVA']] = data.matrix(clr(data_objects[['Fxn.SILVA']] + 0.1))
data_objects[['Fxn.SILVA']] = data_objects[['Fxn.SILVA']][,which(!(colnames(data_objects$Fxn.SILVA) %in% to_drop))]


In [8]:
library(caret)

Loading required package: lattice

Loading required package: ggplot2


Attaching package: ‘caret’


The following object is masked from ‘package:compositions’:

    R2


The following object is masked from ‘package:survival’:

    cluster




In [9]:
tax_df = read.delim('../data/otu_annotations_faprotax_oeu_taxonomy.txt', row.names=1, sep="\t")

In [10]:
set.seed(123)
mixed_data = cbind.data.frame(env_data[,'hypoxia'], ra_rar_df)
colnames(mixed_data) <- c('hypoxia', colnames(ra_rar_df))

# define the control using a random forest selection function
control <- rfeControl(functions=rfFuncs, method="repeatedcv", number=10)
# run the RFE algorithm
results <- rfe(mixed_data[,2:ncol(mixed_data)], mixed_data[,1], sizes=1:10, metric='Accuracy', 
               rfeControl=control)
# summarize the results
results
rfe_imp_scaled <- varImp(results, scale = TRUE)
tax_df[rownames(rfe_imp_scaled)[which(rfe_imp_scaled > 2.5)],1:7]
cor(as.integer(mixed_data$hypoxia), mixed_data[,rownames(rfe_imp_scaled)[which(rfe_imp_scaled > 2.5)]])


Recursive feature selection

Outer resampling method: Cross-Validated (10 fold, repeated 1 times) 

Resampling performance over subset size:

 Variables Accuracy  Kappa AccuracySD KappaSD Selected
         1   0.8800 0.6052    0.04767  0.1758         
         2   0.8969 0.6585    0.04640  0.1520         
         3   0.9096 0.6967    0.04746  0.1713         
         4   0.9097 0.7075    0.06747  0.2268         
         5   0.9222 0.7395    0.05977  0.2205         
         6   0.9270 0.7671    0.06359  0.2272         
         7   0.9355 0.7865    0.06096  0.2279         
         8   0.9357 0.7864    0.05719  0.2209         
         9   0.9398 0.7967    0.05717  0.2229        *
        10   0.9315 0.7774    0.06017  0.2230         
      1349   0.9311 0.7548    0.05763  0.2280         

The top 5 variables (out of 9):
   OTU31, OTU629, OTU48, OTU388, OTU334


,Kingdom,Phylum,Class,Order,Family,Genus,Species
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
OTU31,Bacteria,Proteobacteria,Alphaproteobacteria,Rhodobacterales,Rhodobacteraceae,,
OTU629,Bacteria,Proteobacteria,Alphaproteobacteria,,,,
OTU388,Bacteria,Proteobacteria,Alphaproteobacteria,Rhodospirillales,Magnetospiraceae,,
OTU48,Bacteria,Proteobacteria,Gammaproteobacteria,Chromatiales,Sedimenticolaceae,Sedimenticola,selenatireducens
OTU408,Bacteria,Proteobacteria,Deltaproteobacteria,Bdellovibrionales,Bdellovibrionaceae,OM27_clade,
OTU334,Bacteria,Proteobacteria,Alphaproteobacteria,Rhodospirillales,Rhodospirillaceae,,
OTU545,Bacteria,Proteobacteria,Alphaproteobacteria,Rhodobacterales,Rhodobacteraceae,Maritimibacter,
OTU394,Bacteria,Proteobacteria,Alphaproteobacteria,NRL2,,,
OTU546,Bacteria,Planctomycetes,OM190,,,,


OTU31,OTU629,OTU388,OTU48,OTU408,OTU334,OTU545,OTU394,OTU546,OTU164,OTU355,OTU917,OTU181,OTU559,OTU81,OTU2,OTU562,OTU710,OTU454
0.7030752,0.5691362,0.7152066,0.7310858,0.5391074,0.6777899,0.6549653,0.5042894,0.6024412,0.6725512,0.6488912,0.4632418,0.6424976,0.5880961,0.5574683,0.6953427,0.5236502,0.5880536,0.6205925


In [13]:

write.table(rfe_imp_scaled, file = "/Volumes/KeithSSD/ChesapeakeMicrobiome/data/tosarah/OTU_random_forest_variable_importance_hypoxia.txt", row.names = T)

In [14]:
set.seed(123)
mixed_data_tf = cbind.data.frame(env_data[,'hypoxia'], data_objects[['Taxa.Family']])
colnames(mixed_data_tf) <- c('hypoxia', colnames(data_objects[['Taxa.Family']]))
results <- rfe(mixed_data_tf[,2:ncol(mixed_data_tf)], mixed_data_tf[,1], sizes=4:13, 
               metric='Accuracy', rfeControl=control)
# summarize the results
results
rfe_imp_scaled <- varImp(results, scale = TRUE)
head(rfe_imp_scaled)
cor(as.integer(mixed_data_tf$hypoxia), mixed_data_tf[,rownames(rfe_imp_scaled)[which(rfe_imp_scaled > 3)]])


Recursive feature selection

Outer resampling method: Cross-Validated (10 fold, repeated 1 times) 

Resampling performance over subset size:

 Variables Accuracy  Kappa AccuracySD KappaSD Selected
         4   0.9186 0.7257    0.05703  0.2231         
         5   0.9094 0.7173    0.06845  0.2041         
         6   0.9185 0.7303    0.06729  0.2416         
         7   0.9270 0.7512    0.06278  0.2366         
         8   0.9272 0.7499    0.06260  0.2373        *
         9   0.9272 0.7499    0.06260  0.2373         
        10   0.9272 0.7506    0.05915  0.2277         
        11   0.9139 0.7036    0.06285  0.2403         
        12   0.9224 0.7306    0.06220  0.2414         
        13   0.9137 0.7038    0.06623  0.2489         
       477   0.9188 0.7004    0.05397  0.2212         

The top 5 variables (out of 8):
   Magnetococcales.Magnetococcaceae, Chromatiales.Sedimenticolaceae, Clostridiales.Family_XII, Chromatiales.Chromatiaceae, Thiomicrospirales.Thiomicrospiraceae


,Overall
,<dbl>
Magnetococcales.Magnetococcaceae,8.215634
Chromatiales.Sedimenticolaceae,5.767022
Clostridiales.Family_XII,5.393779
Chromatiales.Chromatiaceae,5.170746
Thiomicrospirales.Thiomicrospiraceae,5.021238
Puniceispirillales.EF100.94H03,5.004713


Magnetococcales.Magnetococcaceae,Chromatiales.Sedimenticolaceae,Clostridiales.Family_XII,Chromatiales.Chromatiaceae,Thiomicrospirales.Thiomicrospiraceae,Puniceispirillales.EF100.94H03,Clostridiales.Peptostreptococcaceae,Desulfovibrionales.Desulfovibrionaceae,Methylococcales.Methylomonaceae,Chitinivibrionales.Chitinivibrionaceae,Syntrophobacterales.Syntrophaceae,Sphingobacteriales.AKYH767,Chloroplast.Chloroplast,Bacteroidales.Marinifilaceae,Betaproteobacteriales.Gallionellaceae
0.7087023,0.518342,0.5386179,0.3974207,0.5283979,0.3968124,0.4844212,0.5408667,0.5079078,0.5091169,-0.2146982,-0.3056477,-0.5336895,0.4437231,0.4285922


In [15]:
write.table(rfe_imp_scaled, file = "/Volumes/KeithSSD/ChesapeakeMicrobiome/data/tosarah/TaxonomicFams_random_forest_variable_importance_hypoxia.txt", row.names = T)

In [16]:
set.seed(123)
mixed_data_oeu = cbind.data.frame(env_data[,'hypoxia'], data_objects[['Fxn.SILVA']] )
colnames(mixed_data_oeu) <- c('hypoxia', colnames(data_objects[['Fxn.SILVA']]))

results <- rfe(mixed_data_oeu[,2:ncol(mixed_data_oeu)], mixed_data_oeu[,1], sizes=5:15, 
               metric='Accuracy', rfeControl=control)
# summarize the results
results

rfe_imp_scaled <- varImp(results, scale = TRUE)
head(rfe_imp_scaled)
cor(as.integer(mixed_data_oeu$hypoxia), mixed_data_oeu[,rownames(rfe_imp_scaled)[which(rfe_imp_scaled > 3)]])


Recursive feature selection

Outer resampling method: Cross-Validated (10 fold, repeated 1 times) 

Resampling performance over subset size:

 Variables Accuracy  Kappa AccuracySD KappaSD Selected
         5   0.9225 0.7607    0.06752  0.1943        *
         6   0.9185 0.7426    0.06594  0.1970         
         7   0.9185 0.7349    0.06594  0.2050         
         8   0.9143 0.7158    0.05410  0.1822         
         9   0.9143 0.7231    0.06138  0.1910         
        10   0.9100 0.7139    0.06299  0.1903         
        11   0.9143 0.7286    0.05815  0.1698         
        12   0.9101 0.7096    0.05944  0.1831         
        13   0.9052 0.6852    0.05350  0.1659         
        14   0.9141 0.7230    0.06123  0.1909         
        15   0.8973 0.6412    0.06398  0.2445         
        51   0.9139 0.6909    0.04940  0.1836         

The top 5 variables (out of 5):
   chloroplasts, thiosulfate_respiration, sulfur_respiration, denitrification, methanotrophy


,Overall
,<dbl>
chloroplasts,10.960774
thiosulfate_respiration,9.652290
methanotrophy,9.523785
sulfur_respiration,9.329692
denitrification,9.204552
anoxygenic_photoautotrophy,8.729257


chloroplasts,thiosulfate_respiration,methanotrophy,sulfur_respiration,denitrification,anoxygenic_photoautotrophy
-0.5324805,0.5177664,0.4734762,0.479624,-0.1745629,0.2685986


In [17]:
write.table(rfe_imp_scaled, file = "/Volumes/KeithSSD/ChesapeakeMicrobiome/data/tosarah/FAPROTAX_Fxns_random_forest_variable_importance_hypoxia.txt", row.names = T)